# AI Rapper System - Google Colab Training

## Setup Instructions:
1. Upload this notebook to Google Colab
2. Go to Runtime > Change runtime type > Select GPU
3. Run each cell in order
4. Download your trained model at the end

**Estimated Time:** 2-4 hours

## Step 1: Upload Training Data

In [ ]:
from google.colab import files
import json

print("📁 Upload your training_lyrics.json file:")
uploaded = files.upload()

# Verify upload
with open('training_lyrics.json', 'r') as f:
    data = json.load(f)
    print(f"\n✅ Loaded {len(data['training_data'])} verses for training")

## Step 2: Install Dependencies

In [ ]:
print("📦 Installing required packages...\n")
!pip install -q transformers datasets accelerate torch
print("\n✅ Dependencies installed!")

## Step 3: Prepare Training Script

In [ ]:
%%writefile train_model.py
import json
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset

print("🎤 AI Rapper Training Script\n")

# Load training data
with open('training_lyrics.json', 'r') as f:
    data = json.load(f)

# Prepare training texts
texts = []
for item in data['training_data']:
    text = f"Prompt: {item['prompt']}\n\nLyrics:\n{item['lyrics']}\n\n"
    texts.append(text)

print(f"📊 Prepared {len(texts)} training examples\n")

# Load model and tokenizer
print("🔧 Loading GPT-2 model...")
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set pad token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# Tokenize
print("📝 Tokenizing texts...")
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

dataset = Dataset.from_dict({'text': texts})
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])
tokenized_dataset.set_format('torch')

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    learning_rate=5e-5,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train
print("\n🚀 Starting training...\n")
trainer.train()

# Save model
print("\n💾 Saving trained model...")
model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_model')
print("\n✅ Training complete!")

## Step 4: Train the Model (This takes 2-4 hours)

In [ ]:
!python train_model.py

## Step 5: Test the Model

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load trained model
model = GPT2LMHeadModel.from_pretrained('./trained_model')
tokenizer = GPT2Tokenizer.from_pretrained('./trained_model')

# Generate test lyrics
prompt = "Prompt: Write aggressive battle rap bars\n\nLyrics:\n"
inputs = tokenizer(prompt, return_tensors='pt')

outputs = model.generate(
    inputs['input_ids'],
    max_length=200,
    temperature=0.9,
    top_p=0.95,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("🎤 Generated Lyrics:\n")
print(generated_text)

## Step 6: Download Your Trained Model

In [ ]:
# Zip the model for download
!zip -r trained_model.zip ./trained_model

# Download
from google.colab import files
print("📥 Downloading your trained model...")
files.download('trained_model.zip')
print("\n✅ Download complete! Extract this on your local machine.")

## Next Steps:

1. Extract `trained_model.zip` on your computer
2. Place the `trained_model` folder in your project's `models/` directory
3. Update `.env`: `LOCAL_MODEL_PATH=./models/trained_model`
4. Restart your API server
5. Generate lyrics with YOUR trained model!

**Optional:** Convert to GGUF for faster CPU inference:
```bash
# Install llama.cpp tools
# Convert model to GGUF format
```